# 0. Importing packages

In [7]:
# Load the autoreload extension to automatically reload modules before executing code (to avoid restarting the kernel)
%load_ext autoreload 

# Enable autoreload for all modules
%autoreload 2

# Python 
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import json

# Custom Packages
from resources.github_functions import GithubScraper

GitHub access token collected from config: gith...
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0.1 Set file paths ###

In [ ]:
# File paths
import resources.filepaths as fp

fp_main = fp.fp_main
fp_main_output = fp.fp_main_output

# To output data that has to go to external s-drive
fp_main_external = fp.fp_main_external
fp_output_external = fp.fp_output_external

# 1. Load in the initial company list

## 1.2 Processing the company list

In [3]:
# Load the initial list of companies
initial_list = pd.read_csv(
    fp_main_output / "initial_list.csv"
)

# Subset relevant company info for initial list
list_of_company_info = initial_list[
    ['company_search_keyword', 'company_category', 'without_location_filter', 'company_label_name']
]

# Create company category map for initial list
company_category_map = dict(zip(
    list_of_company_info['company_search_keyword'],
    list_of_company_info['company_category']
))

# Output company category map
with open(fp_main_output / "company_category_map.jsonl", 'w', encoding='utf-8') as f:
    for keyword, category in company_category_map.items():
        f.write(json.dumps({'company_search_keyword': keyword, 'company_category': category}, ensure_ascii=False) + '\n')

In [4]:
# Create a zip object to loop through, when querying companies.
company_location_filter_bool_zip = zip(
    list_of_company_info['company_search_keyword'],
    list_of_company_info['without_location_filter'],
    list_of_company_info['company_label_name']
)

# 2. Queriyng the company names and scraping users

## 2.1 Loading in scrapelogs

In [5]:
# Collecting users already scraped
users_already_scraped = set()
companies_already_scraped = set()

user_log_file_name = 'first_tier_user_scrapelog.jsonl'
company_log_file_name = 'company_scrapelog.jsonl'
user_log_path = fp_output_external / user_log_file_name
company_log_path = fp_main_output / company_log_file_name

# Ensure files exist and print message
if user_log_path.exists():
    print(f"[INFO] User log exists: {user_log_path.name}")
else:
    user_log_path.touch(exist_ok=True)

if company_log_path.exists():
    print(f"[INFO] Company log exists: {company_log_path.name}")
else:
    company_log_path.touch(exist_ok=True)

# Read user log
with open(user_log_path, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            user_info = json.loads(line)
            users_already_scraped.add(user_info['user_login'])
        except (json.JSONDecodeError, KeyError) as err:
            print(f"[WARNING] Skipping malformed user line: {err}")

# Read company log
with open(company_log_path, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            company = json.loads(line)
            companies_already_scraped.add(company['company_name'])
        except (json.JSONDecodeError, KeyError) as err:
            print(f"[WARNING] Skipping malformed company line: {err}")

[INFO] User log exists: first_tier_user_scrapelog.jsonl
[INFO] Company log exists: company_scrapelog.jsonl


## 2.2 Instantiating the GithubScraper and scrape

In [6]:
# 1. Create instance of GithubScraper
gs = GithubScraper(
    users_already_scraped=users_already_scraped,
    companies_already_scraped=companies_already_scraped,
    repo_limit=50
)
print(f'GitHub REST API ratelimit reset time for token is {gs.reset_time_point}. '
      f'That will be in a little less than {gs.reset_time_in_minutes} minutes.')

# 2. Define output file name
file_name = 'first_tier_userinfo'

# 3. Loop through company queries
for search_query, skip_location_filter, company_label in tqdm(company_location_filter_bool_zip, total=len(initial_list), unit='company'):

    # 3.1 Skip company if already scraped
    if company_label in gs.companies_already_scraped:
        print(f'[INFO] Company {company_label} already scraped. Skipping.')
        continue

    print(f'[INFO] Scraping users for company: {company_label}')
    
    # 3.2 Get users for this company
    users = gs.get_gh_users(search_query, skip_location_filter)

    # 3.3 Loop through users
    for named_user, company in users:

        # 3.3.1 Skip user if already scraped
        if named_user.login in gs.users_already_scraped:
            print(f'[INFO] User {named_user.login} already scraped. Skipping.')
            continue

        print(f'[INFO] Scraping user: {named_user.login}')
        gs.users_already_scraped.add(named_user.login)

        # 3.3.2 Get user info (may return None if repo limit exceeded or no match)
        user_row = gs.get_user_info(named_user, company_label, company_filter=True)
        if user_row is None:
            continue  # Skip user if they don't meet scraping criteria

        # 3.3.3 Extract match data
        location_match = user_row.matched_location
        inferred_company = user_row.inferred_company
        matched_company_strings = user_row.matched_company_strings

        # 3.3.4 Save user info and log result
        gs.save_file(user_row, file_name, remove_existing_file=True)
        gs.log_user_w_match(named_user.login, inferred_company, matched_company_strings, location_match, user_log_path) # type: ignore

        print(f'[INFO] {gs.USERS_SCRAPED} users scraped so far.')

    # 3.4 Log company after scraping all users
    gs.log_company(company_label, company_log_path) # type: ignore

GithubScraper initialized with 8 companies and 38 users already scraped.
GitHub REST API ratelimit reset time for token is 2025-08-12 15:41:28. That will be in a little less than 55 minutes.


  0%|          | 0/33 [00:00<?, ?company/s]

[INFO] Company nodes already scraped. Skipping.
[INFO] Company abtion already scraped. Skipping.
[INFO] Company heyday already scraped. Skipping.
[INFO] Company trifork already scraped. Skipping.
[INFO] Company frontit already scraped. Skipping.
[INFO] Company holion already scraped. Skipping.
[INFO] Company kruso already scraped. Skipping.
[INFO] Company pandiweb already scraped. Skipping.
[INFO] Scraping users for company: uptime
[NEW] GitHub ratelimit threshold set to 300 (max rate: 5000)
[INFO] User andersmandersen already scraped. Skipping.
[INFO] User uptime-development already scraped. Skipping.
[INFO] Scraping user: uptimedk
[NEW] GitHub ratelimit threshold set to 5 (max rate: 30)
[NEW] GitHub ratelimit threshold set to 300 (max rate: 5000)
User match uptimedk logged.
[INFO] 39 users scraped so far.
[INFO] Scraping user: alertdesk
User match alertdesk logged.
[INFO] 40 users scraped so far.
[INFO] Scraping user: SubleGG


 27%|██▋       | 9/33 [00:34<01:32,  3.87s/company]

User match SubleGG logged.
[INFO] 41 users scraped so far.
Company uptime logged.
[INFO] Scraping users for company: charlie tango
[INFO] Scraping user: thebuilder
[NEW] GitHub ratelimit threshold set to 5 (max rate: 30)
[NEW] GitHub ratelimit threshold set to 300 (max rate: 5000)
[INFO] Scraping user: charlie-tango
User match charlie-tango logged.
[INFO] 42 users scraped so far.
[INFO] Scraping user: kristofferkjelde
User match kristofferkjelde logged.
[INFO] 43 users scraped so far.
[INFO] Scraping user: mariatlund
User match mariatlund logged.
[INFO] 44 users scraped so far.
[INFO] Scraping user: jacobcharlietango


...

0 - no users found
Company kmd logged.
[INFO] Scraping users for company: adform
[NEW] GitHub ratelimit threshold set to 5 (max rate: 30)


 97%|█████████▋| 32/33 [1:15:04<02:29, 149.62s/company]

1 - no users found
Company adform logged.
[INFO] Scraping users for company: proactivedk


100%|██████████| 33/33 [1:15:05<00:00, 136.52s/company]

0 - no users found
Company proactivedk logged.
